# Accuracy of peak counting with 'peak steepness' on simulated survey data with shape noise

---

This notebook reproduces results from this article: https://arxiv.org/abs/1806.05995, the comparison of cosmological parameter predictions with peak height statistics and peak steepness statistics on simulated weak lensing surveys with convergence maps in the presence of shape noise.

See the models themselves in the python file: generalized_peak_counting.py

---
*Author: Dezso Ribli*


### Parameters

In [1]:
D = '/mnt/data/weaklens/columbia_data/'  # data folder

NG = 8. #  Effective number of galaxies for current ground based observations
RES = 2.  # reasonable angular resolution in arcmins for this noise

# NG = 8. #  Effective number of galaxies for LSST/EUCLID
# RES = 2.  # reasonable angular resolution in arcmins for this noise

N_REAL = 10000  # number of random realizations
N_MAP = 37  # number of maps in the survey 1 map is 12.25 degˆ2

### Load modules

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
from glob import glob
import os
import pickle
import random

# module for peak counting included in the github repo
from generalized_peak_counting import PeakCount, rmse_bootstrap

### Load data

Actually only load the filenames and parameters, the data will be read on the fly because of its large volume.

In [4]:
filenames = np.array(sorted(glob(os.path.join(D,'*/*.fits'))))  # filenames

# parse parameters from filenames
sigma_8 = np.array([float(os.path.basename(os.path.dirname(fn))[10:15]) for fn in filenames])
omega_m = np.array([float(os.path.basename(os.path.dirname(fn))[2:7]) for fn in filenames])

### Initialize models, both peak height and steepness

Note the histogram bins at peak steepness, they need to be changed when changing noise or resolution!

In [ ]:
peak_height = PeakCount(peak_counting_method = 'original',
                        shape_noise = True,
                        resolution_arcmin = RES, ng = NG,
                        bins = np.linspace(-0.03,0.19,23))

# when using NG=26 change bins to, bins = np.linspace(0.4,1.1,23)) 
# to reproduce numbers in the paper
peak_steepness = PeakCount(peak_counting_method = 'sobel',
                           shape_noise = True,
                           resolution_arcmin = RES, ng = NG,
                           bins = np.linspace(0.3,1.1,23))   

### Calculate mean histograms and covariances

Note: it takes hours!

In [ ]:
%%time
peak_height.fit(filenames, omega_m, sigma_8) 
peak_steepness.fit(filenames, omega_m, sigma_8)
# save it for later this was the longest step
pickle.dump((peak_height, peak_steepness),open('results/demo_survey_n8.pkl','wb'))

In [ ]:
# load it back if it was done before
peak_height, peak_steepness = pickle.load(open('results/demo_survey_n8.pkl','rb'))

### Predict parameters of random realizations of a 'survey' on maps with fiducial parameters 

Note: it takes hours!

In [ ]:
%%time
# fiducial filenames and params
fiducial_filenames = filenames[(omega_m==0.26) & (sigma_8==0.8)]
omega_m_true, sigma_8_true = np.ones(N_REAL)*0.26, np.ones(N_REAL)*0.8
pho, phs, pso, pss = [np.zeros(N_REAL) for _ in range(4)]  # predictions

random.seed(42)  # reset seed
for j in range(N_REAL):  # loop over realizations
    sample_filenames = random.sample(fiducial_filenames, N_MAP)
    pho[j],phs[j] = peak_height.predict(sample_filenames)
    pso[j],pss[j] = peak_steepness.predict(sample_filenames)
    
# save predictions
pickle.dump((omega_m_true, sigma_8_true, pho, phs, pso, pss),
            open('results/survey_predictions_n8.pkl','wb'))

In [ ]:
# load it back if it was done before
omega_m_true, sigma_8_true, pho, phs, pso, pss = pickle.load(
    open('results/survey_predictions_n8.pkl','rb'))

### Estimate errors

In [11]:
print 'Omega_m predictions errors (RMSE)'
print ' Peak height: ', "%.4f +/- %.4f" % rmse_bootstrap(omega_m_true, pho)
print ' Peak steepness:', "%.4f +/- %.4f" % rmse_bootstrap(omega_m_true, pso)
print
print 'sigma_8 predictions errors (RMSE)'
print ' Peak height: ',  "%.4f +/- %.4f" % rmse_bootstrap(sigma_8_true, phs)
print ' Peak steepness:', "%.4f +/- %.4f" % rmse_bootstrap(sigma_8_true, pss)

Omega_m predictions errors (RMSE)
 Peak height:  0.0439 +/- 0.0008
 Peak steepness: 0.0303 +/- 0.0003

sigma_8 predictions errors (RMSE)
 Peak height:  0.0644 +/- 0.0007
 Peak steepness: 0.0567 +/- 0.0006


# The end